In [2]:
!pip install supabase

import os

from supabase import create_client, Client

# Retrieve Supabase URL and Key from environment variables
SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")

# Initialize the Supabase client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

print("Supabase client initialized.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 3.5 MB/s eta 0:00:00
Supabase client initialized.


In [4]:
response = supabase.table("post_content").select("id, extracted_text").limit(5).execute()
data = response.data

print("Retrieved data:")
print(data)

Retrieved data:
[{'id': 70, 'extracted_text': "A healthy woman is the foundation of a healthy family.\n \nAt Bayer in India, aligned with the Government's #SwasthNariSashaktParivarAbhiyan from September 17 - October 2, we carried out multiple activities to showcase our continued commitment to strengthening women’s health and family wellbeing across India.\n \nWe reached over 700 students through 12 workshops on #MenstrualHygiene and conducted 14 structured counselling sessions for 140 pregnant and lactating women, enabling better health and #nutrition outcomes at the last mile.\n \nWe also conducted 1397 tele-consultations, eye screenings, breast cancer screenings, and pulmonary function tests at 12 Phygital #TelemedicineCentres to enable early detection and timely treatment.\n \nAs part of phase II of our #NutrientGapInitiative, we conducted 16 health camps and 40 health education meetings for 3200 residents in selected areas of Karnataka and Uttar Pradesh during the campaign period.\

In [7]:
%pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=d4caf99c57a967e90ea107b187416e43016076e5abde02037701ac0d73e086e7
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [9]:
from langdetect import detect, LangDetectException

detected_languages = []

for record in data:
    text = record.get('extracted_text', '')
    if text:
        try:
            lang = detect(text)
            detected_languages.append(lang)
        except LangDetectException:
            detected_languages.append('unknown')
    else:
        detected_languages.append('None')

print("Detected languages:")
print(detected_languages)

Detected languages:
['en', 'en', 'en', 'en', 'nl']


In [ ]:
update_results = []

for i, (record, lang) in enumerate(zip(data, detected_languages)):
    if i < 5:
        record_id = record.get('id')
        if record_id is not None:
            update_response = supabase.table('post_content').update({'language': lang}).eq('id', record_id).execute()
            update_results.append({
                    'id': record_id,
                    'language': lang,
                    'status': 'success',
                    'response_data': update_response.data
                })

print("Update process complete.")
print("Update results:")
for result in update_results:
    print(result)